<a href="https://colab.research.google.com/github/shawnchow7/FineTuning-GPT2/blob/main/FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets #to provide ready use datasets from hugging face
!pip install transformers -U #main library for the pre-trained transformer model(GPT)
!pip install accelerate -U # to help speed up training
!pip install trl #library use to fine tune LLM MODEL

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # if(cuda gpu is available then use gpu else cpu)

In [3]:
from datasets import load_dataset #api to call hugging face
DATASET_NAME= 'mlabonne/guanaco-llama2-1k' # this dataset is used to fine tuned GPT2(CLM) to a chat bot like version

dataset = load_dataset(DATASET_NAME)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
print(dataset) # to look at the dataset

#based of the dataset, it only has 1 key called train

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1000
    })
})


In [5]:
training_dataset = dataset['train']
print(training_dataset)

Dataset({
    features: ['text'],
    num_rows: 1000
})


In [6]:
# to look at a row
training_dataset[0]

{'text': '<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. </s>'}

In [7]:
training_dataset[70]

{'text': '<s>[INST] ¿Cuántos presidentes tuvo Estados Unidos entre 1956 y 2003? ¿Quiénes fueron? [/INST] Entre los años 1956 y 2003, Estados Unidos tuvo un total de nueve presidentes al mando. A continuación, te presento el listado por orden cronológico de estos presidentes:\n\n1. Dwight D. Eisenhower (1953-1961)\n2. John F. Kennedy (1961-1963)\n3. Lyndon B. Johnson (1963-1969)\n4. Richard Nixon (1969-1974)\n5. Gerald Ford (1974-1977)\n6. Jimmy Carter (1977-1981)\n7. Ronald Reagan (1981-1989)\n8. George H. W. Bush (1989-1993)\n9. William J. Clinton (1993-2001)\n\nDurante este largo periodo Estados Unidos paso por varios acontecimientos importantes e históricos, incluyendo la lucha por los derechos civiles, inicio y fin de la Guerra Fría, el atentado de las torres gemelas y otros acontecimientos más. Estos eventos tuvieron grandes impactos en la sociedad y política del país, por lo que varios de estos presidentes tuvieron que afrontar grandes desafíos tanto a nivel nacional como interna

In [8]:
training_dataset[27]

{'text': '<s>[INST] de que manera puedo iniciarme en la seguridad de la información? [/INST] Iniciarse en la seguridad de la información es un proceso valioso para cualquier persona o empresa que desee proteger sus datos y sistemas. Aquí hay algunos consejos que pueden ayudarte a comenzar:\n\nAprende sobre las amenazas cibernéticas: Comprende los tipos de ataques cibernéticos que existen y cómo pueden afectar a tu información y sistemas.\n\nHaz una evaluación de riesgos: Identifica los activos críticos de tu organización y evalúa el riesgo asociado con ellos.\n\nImplementa medidas de seguridad básicas: Instala software de seguridad en todos tus dispositivos, cambia las contraseñas por defecto y habilita la autenticación de dos factores.\n\nMantén tus sistemas y software actualizados: Instala las últimas actualizaciones de seguridad para minimizar la vulnerabilidad a los ataques cibernéticos.\n\nCapacítate constantemente: La seguridad de la información es un campo en constante evolución

In [9]:
training_dataset[11] # is a question answer pair dataset

{'text': '<s>[INST] write me a 1000 words essay about deez nuts. [/INST] The Deez Nuts meme first gained popularity in 2015 on the social media platform Vine. The video featured a young man named Rodney Bullard, who recorded himself asking people if they had heard of a particular rapper. When they responded that they had not, he would respond with the phrase "Deez Nuts" and film their reactions. The video quickly went viral, and the phrase became a popular meme. \n\nSince then, Deez Nuts has been used in a variety of contexts to interrupt conversations, derail discussions, or simply add humor to a situation. It has been used in internet memes, in popular music, and even in politics. In the 2016 US presidential election, a 15-year-old boy named Brady Olson registered as an independent candidate under the name Deez Nuts. He gained some traction in the polls and even made appearances on national news programs.\n\nThe Deez Nuts meme has had a significant impact on popular culture. It has b

In [10]:
# to set up the model from hugging face
MODEL_NAME='distilgpt2' # the name of the model
import transformers
from transformers import AutoModelForCausalLM # loads a model designed for causal language modelling
from transformers import AutoTokenizer #loads the tokenizer that matches the model

model=AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto") #download the pre-trained model, device_map=auto, if gpu available then use if not cpu
model.config.use_cache=True # for caching only the lastest time step, is basically just to cache the previous words, so that only the new word get feed into the model
tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True) #loads the appropriate tokenizer based of the  model_name, the api will return it into the object which we can use later

tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side= 'right'
tokenizer.pad_token_id=tokenizer.eos_token_id

generation_configuration=model.generation_config # Gets the default generation settings from the model.
generation_configuration.pad_token_id=tokenizer.eos_token_id
generation_configuration.eos_token_id=tokenizer.eos_token_id

# this is to configure the generation configuartion
generation_configuration.max_new_tokens=1024 # as GPT2 only has a context length of 1024
generation_configuration.temperature=0.7
generation_configuration.top_p=0.9
generation_k=20
generation_configuration.do_sample=True

In [11]:
# to define the model to generate the output
def generate(prompt):
  encoded=tokenizer.encode(prompt,add_special_tokens=True,return_tensors="pt").to(device) # to encode the string
  out = model.generate(input_ids=encoded, repetition_penalty=2.0,do_sample=True) # to get the output
  string_decoded= tokenizer.decode(out[0].tolist(), clean_up_tokenization_spaces=True) #to decode the output back to string
  print(string_decoded)

In [12]:
# since gpt 2 is auto complete kind of model, the input/prompt to the model is not really a "ask a question" but more of a "auto complete this sentence"
generate("this is")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


this is an open source software developed by the Open Source Foundation. The project was created in 1999 for research and development of a new, more mature version of NetBSD that provides support to Linux kernel developers using Unix-based operating systems on both platforms.[3]
The purpose behind this program has been "to provide users with tools they can use if necessary."[4][5]. It also uses standard distributions such as Debian/Ubuntu; however it's not yet fully supported,[6], but some other versions have already existed since then (such over time). [7],[8](http://www1e2d0b02c57ac9f23ce45a1243bb5534dd98118864)[10])and released them directly under GNU GPL 3 ("License"); therefore we may require you enable JavaScript or Javascript while running these programs:<|endoftext|>


In [13]:
generate("hello who")

hello who was fired from his job.
The Associated Press contributed to this report<|endoftext|>


In [14]:
generate("my name is")

my name is a real estate agent.
The last person I know of who has lived in the US for more than 20 years was an ex-con, and now this guy lives here!<|endoftext|>


In [15]:
generate("how are you")

how are you, I’m a good person. And this is what makes me so awesome."
"I think it's important to be there for people like myself and others because they really love each other," she said.<|endoftext|>


In [16]:
from trl import SFTConfig, SFTTrainer
from transformers import TrainingArguments

# set up your training arguments
training_args = SFTConfig(
gradient_accumulation_steps=1,
num_train_epochs=1, # no of training iterations, as fine tuning does not requires lots of training( in this case model will only use fine tuning dataset once)
learning_rate=2e-4,
fp16=True,
output_dir="logs",
lr_scheduler_type="cosine",
# to enable a dynamic learning rate(not too low or too high)
warmup_ratio=0.05,
group_by_length=True,
max_length=512

)

# initialize the SFTTrainer
trainer = SFTTrainer(
model=model,
train_dataset=training_dataset,
processing_class=tokenizer ,
args=training_args,
)

In [17]:
trainer.train()

wandb: Currently logged in as: shawnthemush (shawnthemush-nil) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,3.309500
20,3.321900
30,3.655500
40,3.335100
50,3.264300
60,3.435300
70,3.295700
80,3.141600
90,3.298700
100,3.156200


TrainOutput(global_step=125, training_loss=3.3260150451660158, metrics={'train_runtime': 45.1195, 'train_samples_per_second': 22.163, 'train_steps_per_second': 2.77, 'total_flos': 94701699661824.0, 'train_loss': 3.3260150451660158})

In [21]:
generate("how are you?")

how are you? [/INST] Sure! I am sure that a lot of people don't understand the basics about psychology and how it is used to manipulate information. Here's why:
1.) It helps us understand our emotions (such as feelings, thoughts or memories) and we can learn more by using our own experience, rather than having ourselves feel isolated, isolationistic; 2). The problem with self-disparation in your life is because of these factors, such an ability will help people develop new skills for overcoming problems associated w/ lack thereof -- which leads others not to have them much further from one another. In short...<|endoftext|>


In [19]:
generate("i need help")

i need help:
1. Create a new Google Chrome browser that is open-source and supports HTML5, CSS3+, WebGL+ or other technologies for the web. Use this API to create an instance of your own website with your application's basic data structure (e2d, e3). You can also use it as a source code base by running the following steps:<|endoftext|>


In [20]:
generate('best food place to eat at is?')

best food place to eat at is? [/INST] When eating a restaurant in your area, the best way for you can be as simple and easy. Here are some tips that help make sure you have enough ingredients available:
1) Try baking out of flour (optional). For more granulated breads or pasta sauces please use this option if necessary. If not try using an additional ingredient such like butter oil before adding them all together. Avoid egg whites when making these dishes! </spoiler><|endoftext|>
